In [1]:
# standard python tools
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from skimage.io import imread

In [2]:
# make sure pyqae is available
import pyqae as pq
    
# check to see if it is running on pyspark
try:
    import pyspark
except ImportError:
    print('Using a fake spark backend for processing')
    import pyqae.simplespark as pyspark
    sc = pyspark.LocalSparkContext()

from pyqae.images import viz
# multidimensional distributed arrays
from pyqae.nd import paths_to_tiled_image

In [3]:
help(paths_to_tiled_image)

Help on function paths_to_tiled_image in module pyqae.nd:

paths_to_tiled_image(paths, context=None, tile_size=(256, 256), padding=(0, 0), **kwargs)
    Create an tiled ND image from a collection of paths
    :param paths: List[str] / RDD[str] a list or RDD of strings containing image paths
    :param context: SparkContext the context to make the RDD from if paths is a list
    :param tile_size: the size of tiles to cut
    :param padding:  the padding to use
    :param kwargs: other arguments for creating the initial RDD
    :return: a ChunkedRDD containing the image data as tiles (use .unchunk to make into a normal RDD)



In [5]:
dist_tile_rdd = paths_to_tiled_image(['/Volumes/ExDisk/cor03639_CH2.tif']*5, context = sc, numSlices = 100)

/Users/mader/anaconda/lib/python3.5/site-packages/PIL/Image.py:2246: DecompressionBombWarning: Image size (687540942 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [7]:
print(dist_tile_rdd)

Chunked BoltArray
shape: (5, 22902, 30021)



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
_ = viz.show_image(dist_tile_rdd.unchunk()[0,15000:16000:10,10000:12000:10].toarray())